## Setup

Mount Google Drive

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
%matplotlib inline

Paths and variables

In [ ]:
DATA_FILE_PATH = '/content/drive/MyDrive/GOL/data/game_of_life_frames_100000.h5'
H5PY_DATASET_NAME = 'gol_frames'

## Loading and preprocessing

In [ ]:
def DataGenerator():
  with h5py.File(DATA_FILE_PATH, 'r') as f:
    dataset = f[H5PY_DATASET_NAME]
    for i in range(len(dataset)):
      x = np.array(dataset[i:i+3]/255).astype(int)
      y = np.array(dataset[i+3]/255).astype(int)
      yield x, y

In [ ]:
class CustomGen(TimeseriesGenerator):
  def __getitem__(self, idx):
    with h5py.File(DATA_FILE_PATH, 'r') as f:
      dataset = f[H5PY_DATASET_NAME]
      for i in range(len(dataset)):
        x = np.array(dataset[i:i+3]/255).astype(int)
        y = np.array(dataset[i+3]/255).astype(int)
        return x, y

In [ ]:
test = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [ ]:
f = h5py.File(DATA_FILE_PATH, 'r')
dataset = f[H5PY_DATASET_NAME]
dataset = np.array(dataset[:]/255).astype(int)
print(dataset.shape)
a_generator = TimeseriesGenerator(dataset,dataset, 5, batch_size=1)

In [ ]:
len(a_generator)

In [ ]:
 x, y = a_generator[i]
 print(x)

In [ ]:
print(y)

In [ ]:
for i in range(len(a_generator)):
  print(i)
  x, y = a_generator[i]
  prin

In [ ]:
generator = DataGenerator()

inputs = []
outputs = []

for _ in range(2):
  input, output = next(generator)
  inputs.append(input)
  outputs.append(output)

print(len(inputs))
print(len(outputs))

In [ ]:
generator = DataGenerator()

for i in range(2):
  fig, axs = plt.subplots(1,2, figsize=(10, 10))
  input, output = a_generator[i]

  axs[0].imshow(input)
  axs[0].set_title('Input')
  axs[1].imshow(output)
  axs[1].set_title('Output')
  plt.show()